In [1]:
from konlpy.tag import Okt

<br>
<br>

# One-hot Encoding

In [2]:
tokenizer = Okt()  
tokens = tokenizer.morphs("한국의 수도는 서울이다")

print(tokens)


C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


['한국', '의', '수도', '는', '서울', '이다']


In [3]:
# 단어-인덱스 딕셔너리
word_to_index = {}

# 토큰을 인덱스로 변환
for token in tokens:
    if token not in word_to_index.keys():
        word_to_index[token] = len(word_to_index)
        
print(word_to_index)


{'한국': 0, '의': 1, '수도': 2, '는': 3, '서울': 4, '이다': 5}


In [4]:
# 원핫인코딩으로 변환
def convert_ohe(word, word_to_index):
    
    # 벡터를 단어의 개수만큼 0으로 초기화
    vector = [0]*(len(word_to_index))
    
    # 단어의 인덱스 위치에 1 설정
    vector[word_to_index[word]] = 1
    
    return vector

In [5]:
convert_ohe("서울", word_to_index)


[0, 0, 0, 0, 1, 0]

<br>
<br>

# Bag-of-Words

In [6]:
tokenizer = Okt()  
text = "인공지능은 사람의 지능을 기계에 구현하였다"
tokens = tokenizer.morphs(text)

print(tokens)


['인공', '지능', '은', '사람', '의', '지능', '을', '기계', '에', '구현', '하였다']


In [7]:
# 단어-인덱스 딕셔너리
word_to_index = {}

# 토큰을 인덱스로 변환
for token in tokens:
    if token not in word_to_index.keys():
        word_to_index[token] = len(word_to_index)
        
print(word_to_index)


{'인공': 0, '지능': 1, '은': 2, '사람': 3, '의': 4, '을': 5, '기계': 6, '에': 7, '구현': 8, '하였다': 9}


In [8]:
# BoW로 변환
def convert_bow(sentence, word_to_index):
    
    # 벡터를 단어의 개수만큼 0으로 초기화
    vector = [0]*(len(word_to_index))

    # 문장을 토큰으로 분리
    tokenizer = Okt()
    tokens = tokenizer.morphs(sentence)
    
    # 단어의 인덱스 위치에 1 설정
    for token in tokens:
        if token in word_to_index.keys():
            vector[word_to_index[token]] += 1
    
    return vector

In [9]:
convert_bow("인공지능은 사람의 지능을 기계에 구현하였다", word_to_index)


[1, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
convert_bow("인공지능은 기계의 지능을 말한다", word_to_index)


[1, 2, 1, 0, 1, 1, 1, 0, 0, 0]

<br>
<br>

# CountVectorizer

In [11]:
# 토큰을 문자열로 변환
sentence = " ".join(tokens)

print(sentence)


인공 지능 은 사람 의 지능 을 기계 에 구현 하였다


In [12]:
# CountVectorizer의 입력에 맞게 배열로 변경
sentences = []
sentences.append(sentence)

print(sentences)


['인공 지능 은 사람 의 지능 을 기계 에 구현 하였다']


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# 빈도수 기반으로 벡터화
# 1글자도 인식이 되도록 토큰 패턴 변경
cv = CountVectorizer(token_pattern = r"(?u)\b\w+\b")
cv.fit(sentences)

print(cv.vocabulary_)


{'인공': 7, '지능': 8, '은': 4, '사람': 2, '의': 6, '을': 5, '기계': 1, '에': 3, '구현': 0, '하였다': 9}


In [14]:
# CountVectorizer로 변환
def convert_cv(sentence, cv):
    
    # 문장을 토큰으로 분리
    tokenizer = Okt()
    tokens = tokenizer.morphs(sentence)
    
    # 토큰을 문자열로 변환
    sentence = " ".join(tokens)
    
    # CountVectorizer의 입력에 맞게 배열로 변경
    sentences = []
    sentences.append(sentence)
    
    # 벡터 변환
    vector = cv.transform(sentences).toarray()    
    
    return vector

In [15]:
convert_cv("인공지능은 사람의 지능을 기계에 구현하였다", cv)


array([[1, 1, 1, 1, 1, 1, 1, 1, 2, 1]], dtype=int64)

In [16]:
convert_cv("인공지능은 기계의 지능을 말한다", cv)


array([[0, 1, 0, 0, 1, 1, 1, 1, 2, 0]], dtype=int64)

<br>
<br>

# TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "I like animals",
    "I like food",
    "I hate math",
    "I want to study math",
]

# TF-IDF로 벡터화
# 1글자도 인식이 되도록 토큰 패턴 변경
tf_idf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
tf_idf.fit(documents)

print(tf_idf.vocabulary_)


{'i': 3, 'like': 4, 'animals': 0, 'food': 1, 'hate': 2, 'math': 5, 'want': 8, 'to': 7, 'study': 6}


In [18]:
# 다른 문서에도 많이 나온 단어는 낮은 수치
tf_idf.transform(["I like animals"]).toarray()


array([[0.72664149, 0.        , 0.        , 0.37919167, 0.5728925 ,
        0.        , 0.        , 0.        , 0.        ]])

In [19]:
# 같은 문서에 많이 나온 단어는 높은 수치
tf_idf.transform(["I like animals and love animals"]).toarray()


array([[0.90406978, 0.        , 0.        , 0.23589056, 0.3563895 ,
        0.        , 0.        , 0.        , 0.        ]])

<br>
<br>

# TF-IDF로 유사도 비교

In [20]:
import pandas as pd

# 영화 데이터셋 로드
data = pd.read_csv("movies_metadata.csv", low_memory=False)

len(data)


45466

In [21]:
# 첫 번째 데이터 출력
data.head(1)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [22]:
# 첫 번째 데이터의 overview 출력
# 영화에 대한 설명
data.head(1)["overview"][0]


"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [23]:
# 데이터의 일부만 사용
data = data.head(10000)

# Null인 항목의 개수
data["overview"].isnull().sum()


29

In [24]:
# Null인 항목을 빈 값으로 대체
data["overview"] = data["overview"].fillna("")

In [25]:
# Null인 항목이 없어야 TfidfVectorizer() 가능
data["overview"].isnull().sum()


0

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환
tf_idf = TfidfVectorizer(stop_words="english")
tf_idf_matrix = tf_idf.fit_transform(data["overview"])

# 데이터의 개수 : 10000
# 단어의 개수 : 32350
print(tf_idf_matrix.shape)


(10000, 32350)


In [27]:
from sklearn.metrics.pairwise import linear_kernel

# 1000 x 1000을 서로 내적하여 코사인 유사도를 구함
# 각 항목은 두 영화의 유사도를 나타냄
cosine_sim = linear_kernel(tf_idf_matrix, tf_idf_matrix)

print(cosine_sim.shape)


(10000, 10000)


In [28]:
# 중복을 제거하여 영화 제목을 시리즈로 생성
indices = pd.Series(data.index, index=data["title"]).drop_duplicates()

print(indices.head())


title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [29]:
# 유사한 영화를 구함
def get_similar(title, indices, cosine_sim):

    # 영화의 인덱스를 구함
    try:
        index = indices[title]
    except:
        return None
    
    # 해당 영화의 유사도를 배열로 변환
    # 0 : 인덱스, 1 : 유사도
    scores = list(enumerate(cosine_sim[index]))

    # 유사도(x[1] 항목)를 기준으로 높은 순으로 정렬
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # 가장 유사도가 높은 자신을 제외하고 5개를 추출
    scores = scores[1:6]

    # 인덱스를 구함
    indices = [x[0] for x in scores]

    # 각 인덱스의 영화 제목을 구함
    titles = data["title"].iloc[indices] 
    
    return titles

In [30]:
get_similar("Toy Story", indices, cosine_sim)


2997              Toy Story 2
8327                The Champ
1071    Rebel Without a Cause
3057          Man on the Moon
1932                Condorman
Name: title, dtype: object

In [31]:
get_similar("Star Wars", indices, cosine_sim)


1154    The Empire Strikes Back
1167         Return of the Jedi
1267               Mad Dog Time
5187        The Triumph of Love
309           The Swan Princess
Name: title, dtype: object